In [ ]:
# Databricks Real-Time Streaming with Azure Event Hubs → Delta Lake (Bronze/Silver/Gold)
# Generated: 2025-08-20 21:09:06
# Notes:
# - This notebook is code-only (no markdown cells) for clean GitHub rendering.
# - Replace the placeholders below with your actual secrets and paths before running on Databricks.
# - Designed for Databricks Runtime 12+ with Delta enabled.

# Utility: detect if running on Databricks and provide lightweight shims for dbutils to allow local static analysis.
try:
    dbutils  # type: ignore
except NameError:
    class _DBUtilsShim:
        class widgets:
            _vals = {}
            @staticmethod
            def text(name, defaultValue="", label=""):
                _DBUtilsShim.widgets._vals[name] = defaultValue
            @staticmethod
            def dropdown(name, defaultValue, choices, label=""):
                _DBUtilsShim.widgets._vals[name] = defaultValue
            @staticmethod
            def get(name):
                return _DBUtilsShim.widgets._vals.get(name, "")
            @staticmethod
            def removeAll():
                _DBUtilsShim.widgets._vals.clear()
        class fs:
            @staticmethod
            def ls(path): return []
    dbutils = _DBUtilsShim()

In [ ]:
# ------------------------------
# 1) PARAMETERS (Widgets)
# ------------------------------
dbutils.widgets.removeAll()

# Event Hubs
dbutils.widgets.text("eh_connection", "REPLACE_WITH_EVENT_HUBS_CONNECTION_STRING", "Event Hubs Connection String")
dbutils.widgets.text("eh_consumer_group", "$Default", "Consumer Group")
dbutils.widgets.dropdown("eh_start_position", "@latest", ["@latest", "@earliest"], "Start Position")

# Storage paths (ADLS Gen2 mounted paths or abfss:// URIs)
dbutils.widgets.text("bronze_path", "abfss://datalake@REPLACE.dfs.core.windows.net/bronze/iot_events", "Bronze Path")
dbutils.widgets.text("silver_path", "abfss://datalake@REPLACE.dfs.core.windows.net/silver/iot_events", "Silver Path")
dbutils.widgets.text("gold_path",   "abfss://datalake@REPLACE.dfs.core.windows.net/gold/iot_kpis",   "Gold Path")

# Checkpoints (one per stream sink)
dbutils.widgets.text("ckp_bronze", "abfss://datalake@REPLACE.dfs.core.windows.net/_checkpoints/bronze/iot_events", "Bronze Checkpoint")
dbutils.widgets.text("ckp_silver", "abfss://datalake@REPLACE.dfs.core.windows.net/_checkpoints/silver/iot_events", "Silver Checkpoint")
dbutils.widgets.text("ckp_gold",   "abfss://datalake@REPLACE.dfs.core.windows.net/_checkpoints/gold/iot_kpis",   "Gold Checkpoint")

# Table names (optional if you want Hive Metastore/Unity Catalog)
dbutils.widgets.text("tbl_bronze", "iot_raw_bronze", "Table Bronze")
dbutils.widgets.text("tbl_silver", "iot_clean_silver", "Table Silver")
dbutils.widgets.text("tbl_gold",   "iot_kpi_gold", "Table Gold")

params = {k: dbutils.widgets.get(k) for k in [
    "eh_connection","eh_consumer_group","eh_start_position",
    "bronze_path","silver_path","gold_path",
    "ckp_bronze","ckp_silver","ckp_gold",
    "tbl_bronze","tbl_silver","tbl_gold"
]}
print("PARAMETERS LOADED:", {k: ("***" if "connection" in k else v) for k,v in params.items()})

In [ ]:
# ------------------------------
# 2) IMPORTS & SCHEMA
# ------------------------------
from pyspark.sql.functions import col, from_json, to_timestamp, window, current_timestamp, expr, coalesce, lit
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType

# Expected JSON payload from Event Hubs body (utf-8 string):
# {
#   "deviceId": "dev-123",
#   "site": "ams",
#   "temperature": 21.3,
#   "humidity": 0.42,
#   "ts": 1718035200  # epoch seconds (or provide an ISO string and adjust parsing)
# }
raw_schema = StructType([
    StructField("deviceId", StringType()),
    StructField("site", StringType()),
    StructField("temperature", DoubleType()),
    StructField("humidity", DoubleType()),
    StructField("ts", LongType())
])

In [ ]:
# ------------------------------
# 3) READ STREAM FROM EVENT HUBS → BRONZE (DELTA)
# ------------------------------
# Connector config (Event Hubs Spark connector)
# See: https://learn.microsoft.com/azure/event-hubs/event-hubs-spark-structured-streaming

eh_conf = {
    "eventhubs.connectionString": params["eh_connection"],
    "eventhubs.consumerGroup": params["eh_consumer_group"],
    "eventhubs.startingPosition": params["eh_start_position"]
}

# Read stream
raw_stream = (spark.readStream
    .format("eventhubs")
    .options(**eh_conf)
    .load()
)

# The 'body' column is binary. Convert to string for JSON parsing and keep metadata for traceability.
bronze_df = (raw_stream
    .withColumn("body_str", col("body").cast("string"))
    .withColumn("ingest_ts", current_timestamp())
)

# Write to Bronze as Delta (append-only, raw immutable)
bronze_writer = (bronze_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", params["ckp_bronze"])
    .option("path", params["bronze_path"])
    .trigger(processingTime="10 seconds")
)

# Start the Bronze stream (uncomment in Databricks)
# bronze_query = bronze_writer.start()
# display(bronze_query)
print("Bronze stream writer defined. Uncomment start() to run on Databricks.")

In [ ]:
# ------------------------------
# 4) TRANSFORM BRONZE → SILVER (PARSE JSON, CLEANSE, DEDUP)
# ------------------------------
bronze_read = (spark.readStream
    .format("delta")
    .load(params["bronze_path"])
)

parsed = (bronze_read
    .withColumn("json", from_json(col("body_str"), raw_schema))
    .select(
        col("json.deviceId").alias("device_id"),
        col("json.site").alias("site"),
        col("json.temperature").alias("temperature"),
        col("json.humidity").alias("humidity"),
        # prefer event ts if present else use enqueueTime or ingest_ts
        to_timestamp(col("json.ts")).alias("event_time_ts")  # if ts is epoch seconds, use: to_timestamp(col("json.ts"))
    )
    .withColumn("event_time", coalesce(col("event_time_ts"), current_timestamp()))
    .drop("event_time_ts")
)

clean = (parsed
    .filter(col("device_id").isNotNull())
    .withWatermark("event_time", "5 minutes")
    # idempotent dedup if device + event_time used as composite key
    .dropDuplicates(["device_id","event_time"])
)

silver_writer = (clean.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", params["ckp_silver"])
    .option("path", params["silver_path"])
    .trigger(processingTime="10 seconds")
)

# silver_query = silver_writer.start()
# display(silver_query)
print("Silver stream writer defined. Uncomment start() to run on Databricks.")

In [ ]:
# ------------------------------
# 5) AGGREGATE SILVER → GOLD (WINDOWED KPIs)
# ------------------------------
silver_read = (spark.readStream
    .format("delta")
    .load(params["silver_path"])
)

# Example KPIs: per device and 1-minute window → avg temperature, avg humidity, count
kpi = (silver_read
    .withWatermark("event_time", "10 minutes")
    .groupBy(
        window(col("event_time"), "1 minute"),
        col("site"),
        col("device_id")
    )
    .agg(
        expr("avg(temperature) as avg_temperature"),
        expr("avg(humidity) as avg_humidity"),
        expr("count(*) as reading_count")
    )
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        "site", "device_id", "avg_temperature", "avg_humidity", "reading_count"
    )
)

gold_writer = (kpi.writeStream
    .format("delta")
    .outputMode("complete")
    .option("checkpointLocation", params["ckp_gold"])
    .option("path", params["gold_path"])
    .trigger(processingTime="1 minute")
)

# gold_query = gold_writer.start()
# display(gold_query)
print("Gold stream writer defined. Uncomment start() to run on Databricks.")

In [ ]:
# ------------------------------
# 6) OPTIONAL: REGISTER DELTA TABLES
# ------------------------------
spark.sql(f"CREATE TABLE IF NOT EXISTS {params['tbl_bronze']} USING DELTA LOCATION '{params['bronze_path']}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS {params['tbl_silver']} USING DELTA LOCATION '{params['silver_path']}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS {params['tbl_gold']}   USING DELTA LOCATION '{params['gold_path']}'")
print("Tables created (if permissions/catalog configured).")

In [ ]:
# ------------------------------
# 7) OPTIONAL: DATA GENERATOR → EVENT HUBS (RUN AS A ONE-OFF JOB)
# ------------------------------
# Use only if you want to push sample events. Requires azure-eventhub Python package.
# On Databricks: %pip install azure-eventhub
# Then fill in the connection string including EntityPath.
# WARNING: This cell sends data to your Event Hub when executed.

import json, random, time
from datetime import datetime, timezone

try:
    from azure.eventhub import EventHubProducerClient, EventData
    HAVE_EH = True
except Exception as e:
    HAVE_EH = False
    print("azure-eventhub not installed. To enable generator: %pip install azure-eventhub")

eh_conn = params["eh_connection"]  # must include EntityPath
num_batches = 5
events_per_batch = 100

if HAVE_EH and eh_conn.startswith("Endpoint"):
    producer = EventHubProducerClient.from_connection_string(conn_str=eh_conn)
    for b in range(num_batches):
        batch = producer.create_batch()
        for i in range(events_per_batch):
            payload = {
                "deviceId": f"dev-{random.randint(1,5)}",
                "site": random.choice(["ams","utrecht","rotterdam"]),
                "temperature": round(random.uniform(18.0, 30.0), 2),
                "humidity": round(random.uniform(0.3, 0.8), 2),
                "ts": int(time.time())
            }
            batch.add(EventData(json.dumps(payload)))
        producer.send_batch(batch)
        print(f"Sent batch {b+1}/{num_batches}")
        time.sleep(1)
    producer.close()
else:
    print("Generator not executed (missing azure-eventhub or invalid connection string).")

In [ ]:
# ------------------------------
# 8) EXAMPLE ANALYTICS (SQL)
# ------------------------------
spark.sql(f"SELECT * FROM {params['tbl_gold']} ORDER BY window_end DESC LIMIT 20").show(truncate=False)

# Top hot devices (last 30 minutes)
spark.sql(f'''
SELECT device_id, site, avg_temperature, reading_count, window_start, window_end
FROM {params['tbl_gold']}
WHERE window_end > now() - INTERVAL 30 MINUTES
ORDER BY avg_temperature DESC, reading_count DESC
LIMIT 20
''').show(truncate=False)

In [ ]:
# ------------------------------
# 9) STREAM MANAGEMENT HELPERS
# ------------------------------
def stop_all_streams():
    for q in spark.streams.active:
        try:
            print("Stopping:", q.name or q.id)
            q.stop()
        except Exception as e:
            print("Error stopping:", e)

print("Use stop_all_streams() to terminate streams when needed.")